# Part 2: Delaunay Triangulation


In [50]:
import cv2
import numpy as np
import dlib

def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index

def edge_points_inserter(subdiv, shape):
    subdiv.insert((0, 0))                           #insert top left corner
    subdiv.insert((0, int(shape[1]/2)))             #insert top edge middle point
    subdiv.insert((0, shape[1]-1))                  #insert top right corner
    subdiv.insert((int(shape[0]/2), shape[1]-1))    #insert right edge middle point
    subdiv.insert((shape[0]-1, shape[1]-1))         #insert bottom right corner
    subdiv.insert((shape[0]-1, int(shape[1]/2)))    #insert bottom edge middle point
    subdiv.insert((shape[0]-1, 0))                  #insert bottom left corner
    subdiv.insert((int(shape[0]/2), 0))

img = cv2.imread("kimbodnia.png")
img2 = cv2.imread("deniro.jpg")

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
rectangles = detector(gray)
points = predictor(gray, rectangles[0])

gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

#Printing points
#for i in range(0,68):
#    print(points.part(i).x,points.part(i).y)

img_subdiv = cv2.Subdiv2D((0 , 0 , img.shape[0], img.shape[1]))
for i in range(68):
    img_subdiv.insert((points.part(i).x, points.part(i).y))
    
edge_points_inserter(img_subdiv, img.shape)
triangles = img_subdiv.getTriangleList()
triangles1 = np.asarray(triangles) 
#print(triangles1)

points_npyarr = np.zeros((76,2))               #Numpy array of points of face1
for i in range(68):
    points_npyarr[i] = (points.part(i).x, points.part(i).y)
    

shape = img.shape    
points_npyarr[68] = (0,0)                                 
points_npyarr[69] = (0, int(shape[1]/2))
points_npyarr[70] = (0, shape[1]-1)
points_npyarr[71] = (int(shape[0]/2), shape[1]-1)
points_npyarr[72] = (shape[0]-1, shape[1]-1)
points_npyarr[73] = (shape[0]-1, int(shape[1]/2))
points_npyarr[74] = (shape[0]-1, 0)
points_npyarr[75] = (int(shape[0]/2), 0)

#for i in range(76):
#print(points_npyarr[36], points_npyarr[0],points_npyarr[17])
#print(len(points_npyarr))
#print("Triangles type: ",type(triangles))

indexes_triangles = []
for t in triangles:
    pt1 = (t[0], t[1])
    pt2 = (t[2], t[3])
    pt3 = (t[4], t[5])
    index_pt1 = np.where((points_npyarr == pt1).all(axis=1))
    index_pt1 = extract_index_nparray(index_pt1)
    index_pt2 = np.where((points_npyarr == pt2).all(axis=1))
    index_pt2 = extract_index_nparray(index_pt2)
    index_pt3 = np.where((points_npyarr == pt3).all(axis=1))
    index_pt3 = extract_index_nparray(index_pt3)
    if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
        triangle = [index_pt1, index_pt2, index_pt3]
        indexes_triangles.append(triangle)
    cv2.line(img, pt1, pt2, (0, 255, 0), 1)
    cv2.line(img, pt2, pt3, (0, 255, 0), 1)
    cv2.line(img, pt1, pt3, (0, 255, 0), 1)


faces2 = detector(gray2)
for face in faces2:
    landmarks = predictor(gray2, face)
    landmarks_points = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))

shape2 = img2.shape    
landmarks_points.append((0,0))
landmarks_points.append((0, int(shape2[1]/2)))
landmarks_points.append((0, shape2[1]-1))
landmarks_points.append((int(shape2[0]/2), shape2[1]-1))
landmarks_points.append((shape2[0]-1, shape2[1]-1))
landmarks_points.append((shape2[0]-1, int(shape2[1]/2)))
landmarks_points.append((shape2[0]-1, 0))
landmarks_points.append((int(shape2[0]/2), 0))

triangle_list2 = []

for triangle_index in indexes_triangles:
    pt1 = landmarks_points[triangle_index[0]]
    pt2 = landmarks_points[triangle_index[1]]
    pt3 = landmarks_points[triangle_index[2]]
    cv2.line(img2, pt1, pt2, (0, 255, 0), 1)
    cv2.line(img2, pt3, pt2, (0, 255, 0), 1)
    cv2.line(img2, pt1, pt3, (0, 255, 0), 1)
    triangles = [pt1[0], pt1[1], pt2[0], pt2[1], pt3[0], pt3[1]]
    triangle_list2.append(triangles)

array1 = np.asarray(triangle_list2) 

cv2.imshow("Image1", img)
cv2.imshow("image2", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()



# Part 3: Face Morphing

In [52]:
from moviepy.editor import*

img = cv2.imread("kimbodnia.png")
img2 = cv2.imread("deniro.jpg")

def make_homogeneous(triangle):
    #Even indices --> x values
    #Odd indices --> y values
    homogeneous = np.array([triangle[::2],                              
                            triangle[1::2],                             
                            [1, 1, 1]])  
    return homogeneous


def calc_transform(triangle1, triangle2):
    source = make_homogeneous(triangle1).T
    #source
    # P point
    # [x1, y1, 1]
    # [x2, y2, 1]
    # [x3, y3, 1]
    
    target = triangle2
    #target
    # Q point
    # [u1, v1, u2, v2, u3, v3]
    
    Mtx = np.array([np.concatenate((source[0], np.zeros(3))),                     
                    np.concatenate((np.zeros(3), source[0])),                     
                    np.concatenate((source[1], np.zeros(3))),                     
                    np.concatenate((np.zeros(3), source[1])),                     
                    np.concatenate((source[2], np.zeros(3))),                     
                    np.concatenate((np.zeros(3), source[2]))])
    
    coefs = np.matmul(np.linalg.pinv(Mtx), target) #get a11, a12, a13, a21, a22, a23
    Transform = np.array([coefs[:3], coefs[3:], [0, 0, 1]])
    return Transform


def vectorised_Bilinear(coordinates, target_img, size):
    coordinates[0] = np.clip(coordinates[0], 0, size[0]-1)
    coordinates[1] = np.clip(coordinates[1], 0, size[1]-1)
    #in order to eliminate overshoot caused by little errors i have clipped it.
    lower = np.floor(coordinates).astype(np.uint32)
    upper = np.ceil(coordinates).astype(np.uint32)

    error = coordinates - lower    #errors = [a, b]
    resindual = 1 - error #resinduals = [1-a, 1-b]

    top_left = np.multiply(np.multiply(resindual[0], resindual[1]).reshape(coordinates.shape[1],1), target_img[lower[0], lower[1], :])
    top_right = np.multiply(np.multiply(resindual[0], error[1]).reshape(coordinates.shape[1],1), target_img[lower[0], upper[1], :])
    bot_left = np.multiply(np.multiply(error[0], resindual[1]).reshape(coordinates.shape[1],1), target_img[upper[0], lower[1], :])
    bot_right = np.multiply(np.multiply(error[0], error[1]).reshape(coordinates.shape[1],1), target_img[upper[0], upper[1], :])

    return np.uint8(np.round(top_left + top_right + bot_left + bot_right))  #Bilinear interpolation for each transformed point


def image_morph(image1, image2, triangles1, triangles2, transforms, t):
    inter_image_1 = np.zeros(image1.shape).astype(np.uint8)
    inter_image_2 = np.zeros(image2.shape).astype(np.uint8)
    #result = np.zeros(image2.shape)

    for i in range(len(transforms)):
        homo_inter_tri = (1 - t)*make_homogeneous(triangles1[i]) + t*make_homogeneous(triangles2[i])

        polygon_mask = np.zeros(image1.shape[:2], dtype=np.uint8)   #make a given triangular shaped mask
        cv2.fillPoly(polygon_mask, [np.int32(np.round(homo_inter_tri[1::-1, :].T))], color=255)  
        #Fill inside the polygon to create a mask

        seg = np.where(polygon_mask == 255)     #gives x and y coordinates inside the polygon

        mask_points = np.vstack((seg[0], seg[1], np.ones(len(seg[0]))))   #makes column vectors of [x_i, y_i, 1] those obtained points
        #above 2 lines give us the points which will be transformed

        inter_tri = homo_inter_tri[:2].flatten(order="F")   #make it x1 y1 x2 y2 x3 y3 again in order to pass it to the calc_transform function

        inter_to_img1 = calc_transform(inter_tri, triangles1[i]) #Inverse transform from image1 to inter_tri
        inter_to_img2 = calc_transform(inter_tri, triangles2[i]) #Inverse transform from image2 to inter_tri

        mapped_to_img1 = np.matmul(inter_to_img1, mask_points)[:-1]  #float transfered points without ones  (column vectors)
        mapped_to_img2 = np.matmul(inter_to_img2, mask_points)[:-1]  #float transfered points without ones  (column vectors)
        
        inter_image_1[seg[0], seg[1], :] = vectorised_Bilinear(mapped_to_img1, image1, inter_image_1.shape)
        inter_image_2[seg[0], seg[1], :] = vectorised_Bilinear(mapped_to_img2, image2, inter_image_2.shape)
        
    result = (1 - t)*inter_image_1 + t*inter_image_2 
    return result.astype(np.uint8)   

img_triangles = triangles1[:,[1,0,3,2,5,4]]
img2_triangles = array1[:,[1,0,3,2,5,4]]

Transforms = np.zeros((len(img_triangles), 3, 3))
for i in range(len(img_triangles)):
    source = img_triangles[i]          #get source triangle
    target = img2_triangles[i]
    Transforms[i] = calc_transform(source, target)     #A:
    
morphs = []
for t in np.arange(0, 1.0001, 0.02):
    print("processing:\t", t*100, "%")
    morphs.append(image_morph(img, img2, img_triangles, img2_triangles, Transforms, t)[:, :, ::-1])

clip = ImageSequenceClip(morphs,fps=25)
clip.write_videofile('caca.mp4', codec='mpeg4')

processing:	 0.0 %
processing:	 2.0 %
processing:	 4.0 %
processing:	 6.0 %
processing:	 8.0 %
processing:	 10.0 %
processing:	 12.0 %
processing:	 14.000000000000002 %
processing:	 16.0 %
processing:	 18.0 %
processing:	 20.0 %
processing:	 22.0 %
processing:	 24.0 %
processing:	 26.0 %
processing:	 28.000000000000004 %
processing:	 30.0 %
processing:	 32.0 %
processing:	 34.0 %
processing:	 36.0 %
processing:	 38.0 %
processing:	 40.0 %
processing:	 42.0 %
processing:	 44.0 %
processing:	 46.0 %
processing:	 48.0 %
processing:	 50.0 %
processing:	 52.0 %
processing:	 54.0 %
processing:	 56.00000000000001 %
processing:	 57.99999999999999 %
processing:	 60.0 %
processing:	 62.0 %
processing:	 64.0 %
processing:	 66.0 %
processing:	 68.0 %
processing:	 70.0 %
processing:	 72.0 %
processing:	 74.0 %
processing:	 76.0 %
processing:	 78.0 %
processing:	 80.0 %
processing:	 82.0 %
processing:	 84.0 %
processing:	 86.0 %
processing:	 88.0 %
processing:	 90.0 %
processing:	 92.0 %
processing:

100%|█████████████████████████████████████████████████████████████████████████████████| 52/52 [00:00<00:00, 213.69it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: caca.mp4 

